In [15]:
import tensorflow as tf
from tensorflow import keras
from loguru import logger
from tensorflow.keras.models import Model
import cv2
import numpy as np

In [6]:
train_den = "/home/denmann99/autoencoder/pics/denver1"
train_uch = "/home/denmann99/autoencoder/pics/uchtdorf1"
denver=tf.keras.preprocessing.image_dataset_from_directory(train_den,shuffle=True,
                                                     batch_size=64,image_size=(256, 256),seed=123)
uch=tf.keras.preprocessing.image_dataset_from_directory(train_uch,shuffle=True,
                                                     batch_size=64,image_size=(256, 256),seed=123)
def normalizer(generator):
  normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

  def change_inputs(images, labels):
    x = tf.image.resize(normalization_layer(images),[256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return x, x

  return generator.map(change_inputs)
normalized_denver = normalizer(denver)
normalized_uchtdorf = normalizer(uch)

Found 903 files belonging to 1 classes.
Found 1823 files belonging to 1 classes.


In [28]:
input = tf.keras.layers.Input(shape=(256, 256, 3))

# Encoder
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss="binary_crossentropy")
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 128, 128, 32)      9248      
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 256, 256, 32)      9248

In [31]:
autoencoder.fit(
    x=normalized_denver,
    epochs=5,
    batch_size=64,
    shuffle=True
)

Epoch 1/5
15/15 [==============================] - 7s 422ms/step - loss: 0.6014
Epoch 2/5
15/15 [==============================] - 8s 433ms/step - loss: 0.5993
Epoch 3/5
15/15 [==============================] - 8s 441ms/step - loss: 0.5982
Epoch 4/5
15/15 [==============================] - 7s 408ms/step - loss: 0.5971
Epoch 5/5
15/15 [==============================] - 8s 436ms/step - loss: 0.5968


In [34]:
face = cv2.imread("/home/denmann99/autoencoder/pics/uchtdorf1/uchtdorf/uchtdorf_face2.jpg")
face = cv2.resize(face,(256,256))
face = np.reshape(face, (1,256,256,3))
face = face/255
pred = autoencoder.predict(face)
pred = np.reshape(pred,(256,256,3))
pred = pred*255
cv2.imwrite("/home/denmann99/autoencoder/Capturing.png", pred)

True